In [1]:
import os
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [36]:
# import tensorflow as tf
# print(tf.__version__)
# print(tf.config.list_physical_devices('GPU'))
# print(tf.test.is_built_with_cuda())

2.1.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


## Download dataset 
https://drive.google.com/drive/u/3/folders/1sHh6NvuKX6RB5OytLwf4kaqfQ9svJNDQ

## Load data

In [3]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")

x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
# It's a multi-class classification problem 
class_index = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4,
               'dog': 5, 'frog': 6,'horse': 7,'ship': 8, 'truck': 9}
print(np.unique(y_train))

[0 1 2 3 4 5 6 7 8 9]


![image](https://img-blog.csdnimg.cn/20190623084800880.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3lqcDE5ODcxMDEz,size_16,color_FFFFFF,t_70)

## Data preprocess

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to one-hot encoding (keras model requires one-hot label as inputs)
num_classes = 10
print(y_train[0])
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0])

[9]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


## Build model & training (Keras)

In [6]:
# Builde model
model = Sequential() # Sequential groups a linear stack of layers 
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:])) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Conv2D(filters=32, kernel_size=(3, 3))) # Add Convolution layers
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(MaxPooling2D(pool_size=(4, 4))) # Add Max pooling to lower the sptail dimension

model.add(Flatten()) # Flatten the featuremaps
model.add(Dense(units=512)) # Add dense layer with 512 neurons
model.add(Activation('relu')) # Add Relu activation for non-linearity
model.add(Dense(units=num_classes)) # Add final output layer for 10 classes
model.add(Activation('softmax')) # Add softmax activation to transfer logits into probabilities

# initiate SGD optimizer
opt = keras.optimizers.SGD()

# Compile the model with loss function and optimizer, and evaluate with accuracy
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Setup some hyperparameters
batch_size = 32
epochs = 10

# Fit the data into model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 10s 204us/step - loss: 1.9084 - accuracy: 0.3169 - val_loss: 1.6651 - val_accuracy: 0.4022
Epoch 2/10
50000/50000 [==============================] - 9s 173us/step - loss: 1.4650 - accuracy: 0.4776 - val_loss: 1.3684 - val_accuracy: 0.5170
Epoch 3/10
50000/50000 [==============================] - 8s 163us/step - loss: 1.2992 - accuracy: 0.5403 - val_loss: 1.3236 - val_accuracy: 0.5333
Epoch 4/10
50000/50000 [==============================] - 8s 163us/step - loss: 1.1958 - accuracy: 0.5799 - val_loss: 1.1671 - val_accuracy: 0.5994
Epoch 5/10
50000/50000 [==============================] - 8s 166us/step - loss: 1.1080 - accuracy: 0.6116 - val_loss: 1.2143 - val_accuracy: 0.5796
Epoch 6/10
50000/50000 [==============================] - 8s 164us/step - loss: 1.0302 - accuracy: 0.6411 - val_loss: 1.0955 - val_accuracy: 0.6179
Epoch 7/10
50000/50000 [==============================] - 8s 

In [29]:
y_pred = model.predict(x_test)
print(y_pred.shape) # 10000 samples, each sample with probaility of 10 classes

(10000, 10)


In [30]:
y_pred[0] 

array([1.5060628e-03, 1.9026548e-01, 4.8128277e-04, 9.4750547e-04,
       1.0914757e-04, 3.4468432e-04, 3.5507932e-05, 2.1833794e-04,
       9.2192093e-04, 8.0517006e-01], dtype=float32)

In [31]:
np.argmax(y_pred[0]) # argmax to find the predict class with highest probability. 9=truck

9

In [32]:
y_pred = np.argmax(y_pred, axis=1)

## DO NOT MODIFY CODE BELOW!
**Please screen shot your results and post it on your report**

In [23]:
# y_pred = your_model.predict(x_test)
# y_pred = model.predict(x_test)

In [33]:
assert y_pred.shape == (10000,)

In [35]:
y_test = np.load("y_test.npy")
print("Accuracy of my model on test set: ", accuracy_score(y_test, y_pred))

Accuracy of my model on test set:  0.6781
